# Mistral ماڈلز کے ساتھ تعمیر کرنا

## تعارف

اس سبق میں شامل ہے:
- مختلف Mistral ماڈلز کا جائزہ لینا
- ہر ماڈل کے استعمال کے مقاصد اور مواقع کو سمجھنا
- کوڈ کی مثالیں جو ہر ماڈل کی منفرد خصوصیات کو ظاہر کرتی ہیں


## مسٹرال ماڈلز

اس سبق میں، ہم تین مختلف مسٹرال ماڈلز کا جائزہ لیں گے:
**مسٹرال لارج**، **مسٹرال اسمال** اور **مسٹرال نیمو**۔

یہ تمام ماڈلز گٹ ہب ماڈل مارکیٹ پلیس پر مفت دستیاب ہیں۔ اس نوٹ بک میں موجود کوڈ انہی ماڈلز کو استعمال کرے گا۔ مزید معلومات کے لیے یہاں دیکھیں کہ گٹ ہب ماڈلز کے ذریعے [اے آئی ماڈلز کے ساتھ پروٹوٹائپ کیسے بنائیں](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst)۔


## مسٹرال لارج 2 (2407)
مسٹرال لارج 2 اس وقت مسٹرال کا سب سے جدید ماڈل ہے اور اسے کاروباری اداروں کے لیے تیار کیا گیا ہے۔

یہ ماڈل اصل مسٹرال لارج کا اپ گریڈ ہے اور اس میں یہ خصوصیات شامل ہیں:
-  زیادہ بڑا کانٹیکسٹ ونڈو - 128k بمقابلہ 32k
-  ریاضی اور کوڈنگ کے کاموں میں بہتر کارکردگی - اوسط درستگی 76.9% بمقابلہ 60.4%
-  مختلف زبانوں میں بہتر کارکردگی - شامل زبانیں: انگریزی، فرانسیسی، جرمن، ہسپانوی، اطالوی، پرتگالی، ڈچ، روسی، چینی، جاپانی، کوریائی، عربی اور ہندی۔

ان خصوصیات کے ساتھ، مسٹرال لارج ان کاموں میں بہترین ہے:
- *ریٹریول آگمینٹڈ جنریشن (RAG)* - بڑے کانٹیکسٹ ونڈو کی وجہ سے
- *فنکشن کالنگ* - اس ماڈل میں فنکشن کالنگ کی صلاحیت موجود ہے جس سے بیرونی ٹولز اور APIs کے ساتھ انضمام ممکن ہے۔ یہ کالز ایک ساتھ یا ترتیب وار کی جا سکتی ہیں۔
- *کوڈ جنریشن* - یہ ماڈل Python، Java، TypeScript اور C++ میں کوڈ بنانے میں بہترین ہے۔


### مسترال لارج 2 کا استعمال کرتے ہوئے آر اے جی کی مثال


اس مثال میں، ہم Mistral Large 2 کا استعمال کر رہے ہیں تاکہ ایک ٹیکسٹ ڈاکیومنٹ پر RAG پیٹرن چلایا جا سکے۔ سوال کوریائی زبان میں لکھا گیا ہے اور مصنف کی کالج میں داخلے سے پہلے کی سرگرمیوں کے بارے میں پوچھتا ہے۔

یہ Cohere Embeddings Model کا استعمال کرتا ہے تاکہ ٹیکسٹ ڈاکیومنٹ اور سوال دونوں کی ایمبیڈنگز بنائی جا سکیں۔ اس نمونے میں، faiss Python پیکیج کو وییکٹر اسٹور کے طور پر استعمال کیا گیا ہے۔

جو پرامپٹ Mistral ماڈل کو بھیجا جاتا ہے، اس میں سوالات اور وہ چنکس شامل ہوتے ہیں جو سوال سے ملتے جلتے ہیں۔ اس کے بعد ماڈل قدرتی زبان میں جواب فراہم کرتا ہے۔


In [50]:
pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [51]:
import requests
import numpy as np
import faiss
import os

from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential
from azure.ai.inference import EmbeddingsClient

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = requests.get('https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt')
text = response.text

chunk_size = 2048
chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
len(chunks)

embed_model_name = "cohere-embed-v3-multilingual" 

embed_client = EmbeddingsClient(
        endpoint=endpoint,
        credential=AzureKeyCredential(token)
)

embed_response = embed_client.embed(
    input=chunks,
    model=embed_model_name
)



text_embeddings = []
for item in embed_response.data:
    length = len(item.embedding)
    text_embeddings.append(item.embedding)
text_embeddings = np.array(text_embeddings)


d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

question = "저자가 대학에 오기 전에 주로 했던 두 가지 일은 무엇이었나요?？"

question_embedding = embed_client.embed(
    input=[question],
    model=embed_model_name
)

question_embeddings = np.array(question_embedding.data[0].embedding)


D, I = index.search(question_embeddings.reshape(1, -1), k=2) # distance, index
retrieved_chunks = [chunks[i] for i in I.tolist()[0]]

prompt = f"""
Context information is below.
---------------------
{retrieved_chunks}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""


chat_response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful assistant."),
        UserMessage(content=prompt),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(chat_response.choices[0].message.content)

The author primarily engaged in two activities before college: writing and programming. In terms of writing, they wrote short stories, albeit not very good ones, with minimal plot and characters expressing strong feelings. For programming, they started writing programs on the IBM 1401 used for data processing during their 9th grade, at the age of 13 or 14. They used an early version of Fortran and typed programs on punch cards, later loading them into the card reader to run the program.


## مسٹرال اسمال  
مسٹرال اسمال، مسٹرال فیملی کے ماڈلز میں سے ایک اور ماڈل ہے جو پریمیئر/انٹرپرائز کیٹیگری میں شامل ہے۔ جیسا کہ نام سے ظاہر ہے، یہ ایک اسمال لینگویج ماڈل (SLM) ہے۔ مسٹرال اسمال استعمال کرنے کے فائدے یہ ہیں:  
- لاگت میں بچت، مسٹرال کے بڑے ماڈلز جیسے مسٹرال لارج اور نی مو کے مقابلے میں - 80% تک قیمت میں کمی  
- کم تاخیر - مسٹرال کے بڑے ماڈلز کے مقابلے میں تیز جواب  
- لچکدار - مختلف ماحول میں کم وسائل کے ساتھ آسانی سے ڈپلائے کیا جا سکتا ہے۔  

مسٹرال اسمال ان کاموں کے لیے بہترین ہے:  
- ٹیکسٹ پر مبنی کام جیسے خلاصہ بنانا، جذبات کا تجزیہ اور ترجمہ  
- ایسی ایپلیکیشنز جہاں کم لاگت کی وجہ سے بار بار درخواستیں کی جاتی ہیں  
- کم تاخیر والے کوڈ کے کام جیسے ریویو اور کوڈ کی تجاویز


## Mistral Small اور Mistral Large کا موازنہ

Mistral Small اور Large کے درمیان تاخیر میں فرق دکھانے کے لیے، نیچے دیے گئے سیلز چلائیں۔

آپ کو جواب کے وقت میں 3 سے 5 سیکنڈ کا فرق نظر آنا چاہیے۔ اسی پرامپٹ پر جوابات کی لمبائی اور انداز کا بھی مشاہدہ کریں۔


In [ ]:
import os 
endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-small"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

## میسترال نی مو

اس سبق میں زیر بحث دیگر دو ماڈلز کے مقابلے میں، میسترال نی مو واحد مفت ماڈل ہے جس کے پاس اپاچی2 لائسنس ہے۔

اسے میسترال کے پہلے اوپن سورس ایل ایل ایم، میسترال 7B، کا اپ گریڈ سمجھا جاتا ہے۔

نی مو ماڈل کی کچھ اور خصوصیات یہ ہیں:

- *زیادہ مؤثر ٹوکنائزیشن:* یہ ماڈل عام طور پر استعمال ہونے والے tiktoken کی بجائے Tekken ٹوکنائزر استعمال کرتا ہے۔ اس سے زیادہ زبانوں اور کوڈ پر بہتر کارکردگی حاصل ہوتی ہے۔

- *فائن ٹیوننگ:* بیس ماڈل فائن ٹیوننگ کے لیے دستیاب ہے۔ اس سے ان صورتوں میں زیادہ لچک ملتی ہے جہاں فائن ٹیوننگ کی ضرورت ہو سکتی ہے۔

- *نیٹو فنکشن کالنگ* - میسترال لارج کی طرح، اس ماڈل کو فنکشن کالنگ پر تربیت دی گئی ہے۔ یہ اسے منفرد بناتا ہے کیونکہ یہ اوپن سورس ماڈلز میں سے ایک ہے جس نے یہ صلاحیت حاصل کی۔


## میسترال نی مو

اس سبق میں زیر بحث دیگر دو ماڈلز کے مقابلے میں، میسترال نی مو واحد مفت ماڈل ہے جس کے پاس اپاچی2 لائسنس ہے۔

اسے میسترال کے پہلے اوپن سورس ایل ایل ایم، میسترال 7B، کا اپ گریڈ سمجھا جاتا ہے۔

نی مو ماڈل کی کچھ اور خصوصیات یہ ہیں:

- *زیادہ مؤثر ٹوکنائزیشن:* یہ ماڈل عام طور پر استعمال ہونے والے tiktoken کی بجائے Tekken ٹوکنائزر استعمال کرتا ہے۔ اس سے زیادہ زبانوں اور کوڈ پر بہتر کارکردگی حاصل ہوتی ہے۔

- *فائن ٹیوننگ:* بیس ماڈل فائن ٹیوننگ کے لیے دستیاب ہے۔ اس سے ان صورتوں میں زیادہ لچک ملتی ہے جہاں فائن ٹیوننگ کی ضرورت ہو سکتی ہے۔

- *نیٹو فنکشن کالنگ* - میسترال لارج کی طرح، اس ماڈل کو فنکشن کالنگ پر تربیت دی گئی ہے۔ یہ اسے منفرد بناتا ہے کیونکہ یہ اوپن سورس ماڈلز میں سے ایک ہے جس نے یہ خصوصیت سب سے پہلے حاصل کی۔


### ٹوکنائزرز کا موازنہ

اس مثال میں، ہم دیکھیں گے کہ Mistral NeMo ٹوکنائزیشن کو کس طرح ہینڈل کرتا ہے، Mistral Large کے مقابلے میں۔

دونوں مثالیں ایک ہی پرامپٹ لیتی ہیں، لیکن آپ دیکھیں گے کہ NeMo کم ٹوکنز واپس کرتا ہے جبکہ Mistral Large زیادہ ٹوکنز دیتا ہے۔


In [11]:
pip install mistral-common

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.0/797.0 kB 15.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [12]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "open-mistral-nemo	"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

128


In [13]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "mistral-large-latest"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

135


## سیکھنا یہاں ختم نہیں ہوتا، سفر جاری رکھیں

اس سبق کو مکمل کرنے کے بعد، ہماری [جنریٹیو اے آئی لرننگ کلیکشن](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst) دیکھیں تاکہ آپ اپنی جنریٹیو اے آئی کی معلومات کو مزید بہتر بنا سکیں!



---

**اعلانِ دستبرداری**:  
یہ دستاویز AI ترجمہ سروس [Co-op Translator](https://github.com/Azure/co-op-translator) کے ذریعے ترجمہ کی گئی ہے۔ اگرچہ ہم درستگی کی پوری کوشش کرتے ہیں، براہ کرم آگاہ رہیں کہ خودکار ترجمے میں غلطیاں یا عدم درستگی ہو سکتی ہے۔ اصل دستاویز کو اس کی اصل زبان میں مستند ماخذ سمجھا جانا چاہیے۔ اہم معلومات کے لیے پیشہ ور انسانی ترجمہ تجویز کیا جاتا ہے۔ اس ترجمے کے استعمال سے پیدا ہونے والی کسی بھی غلط فہمی یا غلط تشریح کی ذمہ داری ہم پر نہیں ہوگی۔
